In [8]:
import os
import base64
import google.auth
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Set up the Gmail API credentials
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
CREDENTIALS_FILE = './credentials.json'  # Update with your credentials file

# Set up llama 2 quantized CPU version
# LLAMA2_PATH = 'path/to/llama2_quantized_cpu'  # Update with the path to llama 2 executable

def authenticate_gmail_api():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

def get_emails(service, user_id='me', query=''):
    try:
        response = service.users().messages().list(userId=user_id, q=query).execute()
        messages = response.get('messages', [])
        return messages
    except Exception as e:
        print(f'Error occurred: {e}')
        return None

def get_body(message):
    if 'data' in message['payload']['body']:
        return base64.urlsafe_b64decode(message['payload']['body']['data']).decode('utf-8')
    elif 'parts' in message['payload']:
        # Handle the case where the body is nested within 'parts'
        for part in message['payload']['parts']:
            if 'body' in part and 'data' in part['body']:
                return base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
    else:
        return None


def main():
    # Authenticate with Gmail API
    creds = authenticate_gmail_api()
    service = build('gmail', 'v1', credentials=creds)

    # Get inbox emails
    inbox_emails = get_emails(service, query='in:inbox')

    if inbox_emails:
        for email in inbox_emails:
            message = service.users().messages().get(userId='me', id=email['id']).execute()
            msg_data = message['payload']['headers']

            # Process email information (you can customize this part)
            for data in msg_data:
                if data['name'] == 'Subject':
                    subject = data['value']
                    print(f'Subject: {subject}')

            # Extract email body
            body = get_body(message)
            if body:
                print(f'Body: {body}')

            # Use llama 2 for querying on extracted information (replace with your logic)
            # subprocess.run([LLAMA2_PATH, '--query', subject])

if __name__ == '__main__':
    main()


Subject: Build your wealth on the move with YONO SBI
Body: <html><body><table> <tbody> <tr> <td align="center"> <img src="https://www.sbiyono.sbi/wps/wcm/connect/cf89d00f-8585-4d38-a30a-f73b82479737/YWCM342.jpg?MOD=AJPERES" /> </td> </tr> <tr> <td> <p>This is a system generated mail so please do not reply to this mail. <p>Click <a href="https://deliveryalerts.sbi.co.in/RBIEFSUKM?id=198=ZUQDVQEGUFhbHhETRRdGQxEUGBIQFBRFGUUXEBUQREIQGBNFERZBQUNCERNFF0ZDEQREU1RdRw1YFQcADHADD1FRX0tSWQwdUlQIClMDV1ABBwwAAwQMVw8ZS0YG">here</a> to unsubscribe</p>
</p> <p>Disclaimer: State Bank never sends e-mails and embedded links asking you to update or verify confidential, personal and security details. If you receive such emails/phone calls/SMS, NEVER RESPOND to them and report such matter to the bank at report.phishing@sbi.co.in</p> </td> </tr> </tbody> </table><img id='SBBBB' src='https://deliveryalerts.sbi.co.in/RBIEFSUKM?id=198=f0QDVQEGUFhbHhETRRdGQxEUGBIQFBRFGUUXEBUQREIQGBNFERZBQUNCERNFF0ZDEQREU1RdRw1YF

KeyboardInterrupt: 